In [ ]:
%cd /content/ai-competition-baselines
!python -m src.stage1_binary.infer \
  --config configs/stage1_binary.yaml \
  --data_root "{DATA_ROOT}" \
  --ckpt runs/stage1_binary/best_stage1.pt \
  --test_dir images/test \
  --out submissions/stage1.csv

!head -n 5 submissions/stage1.csv